<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/Wyszukiwarka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai py_expression_eval google-api-python-client

Kod instaluje trzy biblioteki Python za pomocą narzędzia pip:

1. openai - oficjalny pakiet OpenAI umożliwiający komunikację z ich API do przetwarzania języka naturalnego i sztucznej inteligencji

2. py_expression_eval - biblioteka do bezpiecznego parsowania i obliczania wyrażeń matematycznych zapisanych jako tekst

3. google-api-python-client - oficjalny klient API Google dla Pythona, pozwalający na interakcję z różnymi usługami Google jak YouTube, Gmail czy Google Drive

Flaga -q (quiet) oznacza tryb cichy instalacji - pip wyświetli tylko najważniejsze informacje i pominie szczegółowe logi procesu instalacji.

In [2]:
from openai import OpenAI
from googleapiclient.discovery import build
from py_expression_eval import Parser
import re, time, os

from google.colab import userdata

Ten fragment kodu importuje niezbędne moduły i klasy:

1. OpenAI z biblioteki openai - główna klasa do komunikacji z API OpenAI

2. build z googleapiclient.discovery - funkcja tworząca interfejs do korzystania z API Google

3. Parser z py_expression_eval - narzędzie do przetwarzania wyrażeń matematycznych

4. re - wbudowany moduł Pythona do operacji na wyrażeniach regularnych

5. time - moduł dostarczający funkcje związane z czasem

6. os - moduł do operacji na systemie operacyjnym i ścieżkach

7. userdata z Colab - bezpieczne przechowywanie danych wrażliwych

In [3]:
class CFG:
    model = "gpt-4o-mini"
    max_tokens = 2048

Jest to klasa konfiguracyjna CFG zawierająca dwie zmienne:

1. model - określa nazwę modelu GPT do wykorzystania ("gpt-4o-mini")

2. max_tokens - ustawia maksymalną liczbę tokenów (jednostek tekstu) na 2048, co ogranicza długość generowanych odpowiedzi

Taka klasa służy do centralizacji ustawień i ułatwia ich późniejszą modyfikację, ponieważ wartości są zdefiniowane w jednym miejscu zamiast być rozproszone po całym kodzie.

In [4]:
# setup OpenAI API connection
api_key = userdata.get("openai")
client = OpenAI(api_key= api_key)

Te linie kodu konfigurują połączenie z API OpenAI:

1. api_key=user_secrets.get_secret("openaivision") - pobiera klucz API z bezpiecznego magazynu Kaggle. "openaivision" to nazwa zapisanego wcześniej sekretu.

2. client = OpenAI(api_key= api_key) - tworzy obiekt klienta OpenAI używając pobranego klucza API. Ten obiekt będzie używany do wszystkich interakcji z usługami OpenAI.

Jest to bezpieczny sposób inicjalizacji połączenia z API, ponieważ wrażliwy klucz nie jest zapisany w kodzie na stałe.

In [6]:
# setup the search engine
cse_key = userdata.get("googlejson")
cse_id = userdata.get("googleidwyszukiwarki")

Te linie pobierają dwa klucze potrzebne do korzystania z wyszukiwarki Google Custom Search Engine (CSE):

1. cse_key - pobiera klucz API dla Google Custom Search z sekretu nazwanego "google_cse"

2. cse_id - pobiera identyfikator spersonalizowanej wyszukiwarki z sekretu "cse_id"

Wyszukiwarka niestandardowa (Custom Search Engine) pozwala na tworzenie własnych, dostosowanych wyszukiwarek Google z określonymi parametrami i ograniczeniami dotyczącymi przeszukiwanych stron.

Setup CSE: https://programmablesearchengine.google.com/controlpanel/all

# Functions

In [7]:
def search(search_term):
    search_result = ""
    service = build("customsearch", "v1", developerKey= cse_key )
    res = service.cse().list(q=search_term, cx = cse_id, num = 10).execute()
    for result in res['items']:
        search_result = search_result + result['snippet']
    return search_result

Funkcja search wykonuje wyszukiwanie w Google i zbiera wyniki:

1. Przyjmuje parametr search_term - frazę do wyszukania

2. Tworzy pusty string search_result do zbierania wyników

3. Tworzy obiekt usługi wyszukiwania za pomocą funkcji build:
   - "customsearch" - nazwa usługi
   - "v1" - wersja API
   - developerKey - używa wcześniej pobranego klucza

4. Wykonuje wyszukiwanie używając metody cse().list() z parametrami:
   - q - fraza do wyszukania
   - cx - identyfikator wyszukiwarki
   - num=10 - liczba wyników do pobrania

5. Łączy wszystkie fragmenty tekstu (snippety) z wyników w jeden string

6. Zwraca połączony tekst wszystkich znalezionych fragmentów

In [8]:
def generate_answer(prompt):
    response = client.chat.completions.create(
            model = CFG.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes essays."},
                {"role": "user", "content": prompt}
            ],
            max_tokens= CFG.max_tokens, n=1, stop=None )

    essay = response.choices[0].message.content.strip()
    return essay

Funkcja generate_answer tworzy tekst używając modelu GPT:

1. Przyjmuje parametr prompt - tekst opisujący zadanie do wykonania

2. Wysyła żądanie do API OpenAI przez utworzonego wcześniej klienta:
   - model - używa modelu zdefiniowanego w klasie CFG
   - messages - lista wiadomości zawierająca:
     * instrukcję systemową określającą rolę asystenta
     * treść zadania od użytkownika
   - max_tokens - limit długości odpowiedzi z CFG
   - n=1 - liczba wariantów odpowiedzi
   - stop=None - brak specjalnego znaku kończącego generowanie

3. Z odpowiedzi wyciąga wygenerowany tekst, usuwa białe znaki na końcach

4. Zwraca wygenerowany tekst

# Base model

In [9]:
muhprompt = "Who is Kamil Winiarski"
print(muhprompt)

Who is Kamil Winiarski


In [10]:
answer1 = generate_answer(muhprompt)
print(answer1)

As of my last knowledge update in October 2021, I do not have specific information about an individual named Kamil Winiarski. It is possible that he may be a private individual, a professional in a particular field, or someone who has gained recognition after that date.

If you have a specific context or details regarding Kamil Winiarski, such as his profession or achievements, I may be able to help you craft more information or context-based content. Alternatively, I recommend checking the latest information through news sources, social media platforms, or professional networking sites.


# Agent

In [11]:
# update the system prompt
system_prompt2 = """
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things

Option 1: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: "the input to the action, to be sent to the tool"

Option 2: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"

Begin!
"""

In [12]:
def extract_action_and_input(text):
    action_pattern = r"Action: (.+?)\n"
    input_pattern = r"Action Input: \"(.+?)\""
    action = re.findall(action_pattern, text)
    action_input = re.findall(input_pattern, text)
    return action, action_input

Funkcja extract_action_and_input analizuje tekst używając wyrażeń regularnych, aby wyodrębnić akcję i jej parametry:

1. action_pattern = r"Action: (.+?)\n" - wzorzec szukający tekstu zaczynającego się od "Action: " i kończącego się nową linią. Nawiasy () oznaczają część do wyodrębnienia.

2. input_pattern = r"Action Input: \"(.+?)\""  - wzorzec szukający tekstu między "Action Input: \"" a znakiem cudzysłowu

3. re.findall() szuka wszystkich wystąpień tych wzorców w tekście

4. Funkcja zwraca dwie listy:
   - action - znalezione akcje
   - action_input - znalezione parametry akcji


In [13]:
def agentic_answer(prompt):
    messages = [
        { "role": "system", "content": system_prompt2 },
        { "role": "user", "content": prompt },
    ]

    while True:
        response = client.chat.completions.create(
            model = CFG.model, messages=messages, temperature=0, top_p=1,)
        response_text = response.choices[0].message.content
        print(response_text)
        # enforce a wait to prevent the Rate Limit error for free-tier users
        time.sleep(3)

        action, action_input = extract_action_and_input(response_text)
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

Funkcja agentic_answer implementuje interaktywnego agenta, który może wyszukiwać informacje i odpowiadać na pytania:

1. Tworzy listę messages z dwiema wiadomościami początkowymi:
   - system_prompt2 (wcześniej zdefiniowana instrukcja systemowa)
   - pytanie użytkownika (prompt)

2. Wchodzi w pętlę, która:
   - Generuje odpowiedź używając API OpenAI z parametrami:
     * temperature=0 (maksymalna spójność odpowiedzi)
     * top_p=1 (uwzględnia wszystkie prawdopodobne tokeny)
   - Czeka 3 sekundy, aby nie przekroczyć limitu zapytań
   - Wyodrębnia akcję i jej parametry z odpowiedzi
   - W zależności od akcji:
     * "Search" - wykonuje wyszukiwanie
     * "Response To Human" - wyświetla końcową odpowiedź i kończy działanie
   - Dodaje odpowiedź i wynik obserwacji do historii rozmowy

3. Pętla kontynuuje pracę, dopóki agent nie zdecyduje, że ma wystarczające informacje do udzielenia końcowej odpowiedzi.

In [14]:
agentic_answer(muhprompt)

Thought: I need to find information about Kamil Winiarski to provide a comprehensive answer. 
Action: Search
Action Input: "Kamil Winiarski"
Observation:  8921 Followers, 349 Following, 182 Posts - Kamil Winiarski (@kamilwiniarski) on Instagram: "Youtube ViniPlay ✉️ Collab - gtavinipl@gmail.com Twitch ...Driver/Cleaner w Lancaster Land Rover · Experience: Lancaster Land Rover · Location: Slough. View Kamil Winiarski's profile on LinkedIn, a professional ...Plats: 172 66 · 67 kontakter på LinkedIn. Visa Kamil Winiarskis profil på LinkedIn, ett yrkesnätverk med 1 miljard medlemmar.View the profiles of professionals named "Kamil Winiarski" on LinkedIn. There are 10+ professionals named "Kamil Winiarski", who use LinkedIn to exchange ...Kamil Winiarski Slaskie, Poland. Level. 75 · Power Player. 368 XP. No information given. View more info. Currently In-Game. Hearts of Iron IV. Profile Awards ...Sep 28, 2021 ... This button displays the currently selected search type. When expanded it provi